In [2]:
import pandas as pd

In [3]:
# Load in the raw csv file
file_path = r"C:\Users\japar\OneDrive\Desktop\SQL\E-Commerce Customer Data for Behavior Analysis\ecommerce_customer_data_combined.csv"
df = pd.read_csv(file_path)

In [4]:
# Preview
print("Initial shape:", df.shape)
print(df.head())

Initial shape: (500000, 13)
   Customer ID     Purchase Date Product Category  Product Price  Quantity  \
0        46251     9/8/2020 9:38      Electronics             12         3   
1        46251    3/5/2022 12:56             Home            468         4   
2        46251   5/23/2022 18:18             Home            288         2   
3        46251  11/12/2020 13:13         Clothing            196         1   
4        13593  11/27/2020 17:55             Home            449         1   

   Total Purchase Amount Payment Method  Customer Age  Returns  \
0                    740    Credit Card            37      0.0   
1                   2739         PayPal            37      0.0   
2                   3196         PayPal            37      0.0   
3                   3509         PayPal            37      0.0   
4                   3452    Credit Card            49      0.0   

         Customer Name  Age  Gender  Churn  
0  Christine Hernandez   37    Male      0  
1  Christine Her

In [5]:
# Basic cleaning
# Drop duplicates in any row
df = df.drop_duplicates()


In [6]:
# Standardize column names
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

In [7]:
# Convert purchase_date to datetime
df['purchase_date'] = pd.to_datetime(df['purchase_date'], errors = 'coerce')

In [8]:
# Filling missing values in numeric columns (except special cases)
special_handling = ['returns', 'churn', 'age']
numeric_cols = df.select_dtypes(include = ['float64', 'int64']).columns
for col in numeric_cols:
    if col not in special_handling:
        df[col] = df[col].fillna(df[col].mean())

In [9]:
# Filling the payment_method column with 'Credit Card' for NULLS
df['payment_method'] = df['payment_method'].fillna('Credit Card')

In [10]:
# Fill return NULLS with '0'
df['returns'] = df['returns'].fillna(0).astype(int)

In [11]:
# Fill age NULLS with mean and convert to an int
df['age'] = df['age'].fillna(df['age'].mean()).astype(int)

In [12]:
# Fill gender NULLS with 'Male'
df['gender'] = df['gender'].fillna("Male")

In [13]:
# Fill churn NULLS with 1 and convert to an int
df['churn'] = df['churn'].fillna(1).astype(int)

In [14]:
# Feature engineering
# Day of the week from purchase_date
df['day_of_week'] = df['purchase_date'].dt.day_name()

In [15]:
# Finding weekend purchases
df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday'])

In [16]:
# Total value of each transaction
df['total_revenue'] = df['product_price'] * df['quantity']

In [17]:
# Creating spending buckets
df['spend_bucket'] = pd.cut(df['total_revenue'],
                            bins = [0, 50, 150, 300, float('inf')],
                            labels = ['Low', "Medium", 'High', 'Very High'])

In [18]:
# Was the item returned?
df['was_returned'] = df['returns'].apply(lambda x: 'Yes' if x == 1 else 'No')

In [19]:
# Categorize gender to title case
df['gender'] = df['gender'].str.title()

In [20]:
# Classifying periods of the day
df['time_of_day'] = df['purchase_date'].dt.hour
df['time_of_day'] = pd.cut(df['time_of_day'],
                           bins = [-1, 6, 12, 17, 23],
                           labels = ['Night', 'Morning', 'Afternoon', 'Evening'])

In [21]:
output_path = r"C:\Users\japar\OneDrive\Desktop\SQL\E-Commerce Customer Data for Behavior Analysis\ecommerce_customer_data_CLEANED.csv"
df.to_csv(output_path, index=False)

In [22]:
df.columns

Index(['customer_id', 'purchase_date', 'product_category', 'product_price',
       'quantity', 'total_purchase_amount', 'payment_method', 'customer_age',
       'returns', 'customer_name', 'age', 'gender', 'churn', 'day_of_week',
       'is_weekend', 'total_revenue', 'spend_bucket', 'was_returned',
       'time_of_day'],
      dtype='object')